## Carregamento dos Dados

In [7]:
import tensorflow as tf
import numpy as np

train_ds = tf.keras.utils.image_dataset_from_directory(
    "dataset/train",
    labels="inferred", # Os rótulos são definidos a partir dos nomes das subpastas
    label_mode="categorical", # Formato categórico dos rótulos
    batch_size=32, # Lotes de 32 imagens
    image_size=(128, 128), # Redimensiona as imagens para 128x128
    shuffle=True, # Embaralha as imagens para evitar que a rede aprenda padrões de ordenação
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    "dataset/test",
    labels="inferred", # Os rótulos são definidos a partir dos nomes das subpastas
    label_mode="categorical", # Formato categórico dos rótulos
    batch_size=32, # Lotes de 32 imagens
    image_size=(128, 128), # Redimensiona as imagens para 128x128
    shuffle=False, # Não precisa embaralhar nos dados de teste
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    "dataset/validation",
    labels="inferred", # Os rótulos são definidos a partir dos nomes das subpastas
    label_mode="categorical", # Formato categórico dos rótulos
    batch_size=32, # Lotes de 32 imagens
    image_size=(128, 128), # Redimensiona as imagens para 128x128
    shuffle=False, # Não precisa embaralhar nos dados de validação
)

Found 7000 files belonging to 7 classes.
Found 1400 files belonging to 7 classes.
Found 1400 files belonging to 7 classes.


## Normalização

In [8]:
# Camada que normaliza as imagens do intervalo [0, 255] para [0, 1]
normalization_layer = tf.keras.layers.Rescaling(1./255)

# Aplicação a normalização às imagens de cada dataset
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))